In [ ]:
#Translate YOLO to COCO and save as a CSV
import os
import csv

def convert_yolo_to_coco(label_dir, output_file, image_width, image_height):
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]
    coco_data = []

    for label_file in label_files:
        with open(os.path.join(label_dir, label_file), 'r') as f:
            lines = f.readlines()

        for line in lines:
            class_id, x_center, y_center, bbox_width, bbox_height = map(float, line.strip().split())

            # Convert YOLO format to COCO format
            x_min = (x_center - bbox_width / 2) * image_width
            y_min = (y_center - bbox_height / 2) * image_height
            x_max = (x_center + bbox_width / 2) * image_width
            y_max = (y_center + bbox_height / 2) * image_height

            # Append the converted bounding box to the COCO data list
            coco_data.append([label_file, x_min, y_min, x_max, y_max, class_id])

    # Write the COCO data to a CSV file
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['image_id', 'x_min', 'y_min', 'x_max', 'y_max', 'class_id'])
        writer.writerows(coco_data)

    print(f"Conversion completed. CSV file saved as {output_file}")

# Specify the label directory, output CSV file, and the original image dimensions
label_directory = 'C:/Users/rowanconverse/Desktop/data/labelTile/'
output_csv_file = 'C:/Users/rowanconverse/Desktop/data/experttiles.csv'
image_width = 684
image_height = 521

# Call the function to convert YOLO format to COCO format and save as CSV
convert_yolo_to_coco(label_directory, output_csv_file, image_width, image_height)

In [ ]:
import os
import csv
import json

def convert_yolo_to_coco(label_dir, output_file, image_width, image_height):
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]
    coco_data = []

    for label_file in label_files:
        image_id = os.path.splitext(label_file)[0] + '.PNG'
        with open(os.path.join(label_dir, label_file), 'r') as f:
            lines = f.readlines()

        for line in lines:
            class_id, x_center, y_center, bbox_width, bbox_height = map(float, line.strip().split())

            # Convert YOLO format to COCO format
            x_min = (x_center - bbox_width / 2) * image_width
            y_min = (y_center - bbox_height / 2) * image_height
            box_width = bbox_width * image_width
            box_height = bbox_height * image_height

            # Append the converted bounding box to the COCO data list
            coco_data.append({
                'filename': image_id,
                'bbox': [x_min, y_min, box_width, box_height],
                'class_id': class_id
            })

    # Write the COCO data to a CSV file
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['filename', 'bbox', 'class_id'])
        for entry in coco_data:
            writer.writerow([entry['filename'], entry['bbox'], entry['class_id']])

    # Save the COCO data as a JSON file
    json_file = os.path.splitext(output_file)[0] + '.json'
    with open(json_file, 'w') as jsonfile:
        json.dump(coco_data, jsonfile)

    print(f"Conversion completed. CSV file saved as {output_file} . JSON file saved as {json_file}")


# Specify the label directory, output CSV file, and the original image dimensions
label_directory = 'C:/Users/rowanconverse/Desktop/data/labelTile/'
output_csv_file = 'C:/Users/rowanconverse/Desktop/data/experttiles.csv'
image_width = 684
image_height = 521

# Call the function to convert YOLO format to COCO format and save as CSV and JSON
convert_yolo_to_coco(label_directory, output_csv_file, image_width, image_height)


In [ ]:
#COCO to YOLO

#assumes that the train/val/test split already has happened and these csv files are saved separately 

import os
import csv
from PIL import Image

# Function to convert coordinates to YOLO format
def convert_to_yolo_format(image_width, image_height, xmin, ymin, width, height):
    x_center = (xmin + width/2) / image_width
    y_center = (ymin + height/2) / image_height
    yolo_width = width / image_width
    yolo_height = height / image_height
    return x_center, y_center, yolo_width, yolo_height

# Path to the directory containing the CSV files
csv_directory = ''

# Path to the directory where the YOLO format annotations will be saved
yolo_directory = ''

# List to store missing images
missing_images = []

# Process each split (train, val, test)
for split in ['train', 'val', 'test']:
    # Read the annotations from the CSV file
    csv_file = os.path.join(csv_directory, f'{split}.csv')

# Read the CSV file
    with open(os.path.join(csv_directory, csv_file), 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header row

        # Create a dictionary to store the annotations for each image
        annotations = {}

        # Process each annotation in the CSV file
        for row in reader:
            filename, bbox, label = row[0], row[2], row[3]
            bbox = list(map(float, bbox[1:-1].split(',')))
            xmin, ymin, width, height = bbox
            if filename not in annotations:
                annotations[filename] = []
            annotations[filename].append((xmin, ymin, width, height, label))

        # Process each image and its annotations
        for filename, image_annotations in annotations.items():
            # Load the image to get its dimensions
            image_path = os.path.join(csv_directory, filename)
            if not os.path.exists(image_path):
                missing_images.append(filename)
                continue

            image_width, image_height = Image.open(image_path).size

            # Convert each annotation to YOLO format
            yolo_annotations = []
            for xmin, ymin, xmax, ymax, label in image_annotations:
                x_center, y_center, width, height = convert_to_yolo_format(
                    image_width, image_height, float(xmin), float(ymin), float(xmax), float(ymax)
                )
                yolo_annotations.append(f"{label} {x_center} {y_center} {width} {height}")

            # Write the YOLO annotations to a text file
            output_file = f"{yolo_directory}/{split}/{os.path.splitext(filename)[0]}.txt"
            os.makedirs(os.path.dirname(output_file), exist_ok=True)
            with open(output_file, 'w') as outfile:
                outfile.write('\n'.join(yolo_annotations))

# Save the list of missing images
missing_images_file = os.path.join(yolo_directory, 'missing_images.txt')
with open(missing_images_file, 'w') as file:
    file.write("\n".join(missing_images))